# 翻訳モデルの作成
データを入れ替えて、事前学習済みモデルに学習させて分類AIを作ったり、翻訳AIを作ったりするコード(0502 SES2022)

# ライブラリやデータの準備

## 依存ライブラリのインストール

In [1]:
!pip install -qU torch==1.7.1 torchtext==0.8.0 torchvision==0.8.2
!pip install -q transformers==4.4.2 pytorch_lightning==1.2.1 sentencepiece

## 各種ディレクトリ作成

* data: 学習用データセット格納用
* model: 学習済みモデル格納用

In [2]:
!mkdir -p /content/data /content/model

In [3]:
# 事前学習済みモデル
PRETRAINED_MODEL_NAME = "google/mt5-small"

# 転移学習済みモデル
MODEL_DIR = "/content/model"

## データ分割

データセットを90% : 5%: 5% の比率でtrain/dev/testに分割します。

* trainデータ: 学習に利用するデータ
* devデータ: 学習中の精度評価等に利用するデータ
* testデータ: 学習結果のモデルの精度評価に利用するデータ

In [4]:
#データクローン
!git clone https://github.com/roy029/OJT1stCorpus.git
!git clone https://github.com/y-akinobu/corpus_Python-JPN.git

fatal: destination path 'OJT1stCorpus' already exists and is not an empty directory.
fatal: destination path 'corpus_Python-JPN' already exists and is not an empty directory.


In [5]:
import pandas as pd
# data = pd.read_csv("neko.csv", sep=",",  index_col=0)
data = pd.read_csv("/content/corpus_Python-JPN/Corpus-DS/DS_B.tsv", sep="\t",  names=("text", "label"))
data

,text,label
0,start = <A>,そこで、 <A> を開始位置とする
1,start = <A>,そこで、開始位置は <A> とする
2,start = <A>,そこで、始まりは <A> とする
3,start = <A>,そこで、開始は <A> とする
4,start = <A>,そこで、 <A> から始める
...,...,...
20402,<A> = pd . get_dummies ( <A> [ <B> ] ),列 <B> についてダミー変数を生成して、 <A> とする
20403,<A> = pd . get_dummies ( <A> [ <B> ] ),<A> ( データフレーム ) の <B> についてダミー変数を作り、 <A> にする
20404,<A> = pd . get_dummies ( <A> [ <B> ] ),<B> についてダミー変数を作り、 <A> とする
20405,<A> = pd . get_dummies ( <A> [ <B> ] ),<B> についてダミー変数をつくり、 <A> とする


In [6]:
#データフレームdataに任意のタグをくっつける

txt_lst = []
lbl_lst = []
for i, r in data.iterrows():
  # print(r["text"])
  r["text"] = "translate English to Japanese: " + f'{r["text"]}' #先頭にタグ：翻訳
  # r["text"] = "advice: " + f'{r["text"]}' #先頭にタグをくっつける
  txt_lst.append(r["text"])

  # r["label"] = "label: " + f'{r["label"]}' #先頭にタグをくっつける
  lbl_lst.append(r["label"])

import pandas as pd

dim2list = [[txt_lst[i], lbl_lst[i]] for i in range(len(lbl_lst))]
data = pd.DataFrame(dim2list, columns=['text', 'label'])
# data.to_csv("data.csv", index=False)
data

,text,label
0,translate English to Japanese: start = <A>,そこで、 <A> を開始位置とする
1,translate English to Japanese: start = <A>,そこで、開始位置は <A> とする
2,translate English to Japanese: start = <A>,そこで、始まりは <A> とする
3,translate English to Japanese: start = <A>,そこで、開始は <A> とする
4,translate English to Japanese: start = <A>,そこで、 <A> から始める
...,...,...
20402,translate English to Japanese: <A> = pd . get_...,列 <B> についてダミー変数を生成して、 <A> とする
20403,translate English to Japanese: <A> = pd . get_...,<A> ( データフレーム ) の <B> についてダミー変数を作り、 <A> にする
20404,translate English to Japanese: <A> = pd . get_...,<B> についてダミー変数を作り、 <A> とする
20405,translate English to Japanese: <A> = pd . get_...,<B> についてダミー変数をつくり、 <A> とする


In [7]:
all_data=[]
def to_all_data(df):
    for item in zip(df['text'], df['label']):
      text = item[0]
      label = item[1]
      # print("ラベル", lbl)
    # if len(text) > 0 and len(label) > 0:
      all_data.append({"text": text, "label": label})
            
#to_all_data("data/hagemasi_corpus_T5.tsv")
to_all_data(data)
#to_all_data("ph_random_tag.tsv")

In [8]:
all_data[2:4]

[{'label': 'そこで、始まりは <A> とする',
  'text': 'translate English to Japanese: start = <A>'},
 {'label': 'そこで、開始は <A> とする',
  'text': 'translate English to Japanese: start = <A>'}]

In [9]:
import random
from tqdm import tqdm

random.seed(2434)
random.shuffle(all_data)

def to_line(data):
    text = data["text"]
    label = data["label"]

    # assert len(text) > 0 and len(label) > 0
    return f"{text}\t{label}\n"

data_size = len(all_data)
print(data_size)

train_ratio, dev_ratio, test_ratio = 0.9, 0.05, 0.05

with open(f"data/train.tsv", "w", encoding="utf-8") as f_train, \
    open(f"data/dev.tsv", "w", encoding="utf-8") as f_dev, \
    open(f"data/test.tsv", "w", encoding="utf-8") as f_test:
    
    for i, data in tqdm(enumerate(all_data)):
        line = to_line(data)
        if i < train_ratio * data_size:
            f_train.write(line)
        elif i < (train_ratio + dev_ratio) * data_size:
            f_dev.write(line)
        else:
            f_test.write(line)

20407


20407it [00:00, 504923.76it/s]


作成されたデータを確認します。

形式: {タイトル}\t{本文}\t{ジャンルID}

In [10]:
{'text': "advice:typeerror: 'tuple' object does not support item assignment", 'hagemashi': 'tuple型は要素の変更ができません。'}

{'hagemashi': 'tuple型は要素の変更ができません。',
 'text': "advice:typeerror: 'tuple' object does not support item assignment"}

In [11]:
print(all_data[1])

{'text': "translate English to Japanese: markeredgecolor = 'gray'", 'label': 'そこで、マーカーのエッジ色としてグレーを使う'}


In [12]:
!head -3 data/test.tsv

translate English to Japanese: pd . merge ( <A> , <B> , left_index = True , right_on = <C> )	左側のデータの位置と右側のデータの <C> コラムをキーとして、 <A> と <B> 表を一つにまとめる
translate English to Japanese: plt . hist ( <A> )	<A> の柱状グラフを作画する
translate English to Japanese: <A> . drop_duplicates ( inplace = True )	表 <A> 中の重複している行を消して、置き換える


# 学習に必要なクラス等の定義

学習にはPyTorch/PyTorch-lightning/Transformersを利用します。

In [13]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl


from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    AutoTokenizer,
    get_linear_schedule_with_warmup
)

# 乱数シードの設定
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

In [14]:
# GPU利用有無
USE_GPU = torch.cuda.is_available()

# 各種ハイパーパラメータ
args_dict = dict(
    data_dir="/content/data",  # データセットのディレクトリ
    model_name_or_path=PRETRAINED_MODEL_NAME,
    tokenizer_name_or_path=PRETRAINED_MODEL_NAME,

    learning_rate=3e-4,
    weight_decay=0.0,
    adam_epsilon=1e-8,
    warmup_steps=0,
    gradient_accumulation_steps=1,

    # max_input_length=64,
    # max_target_length=512,
    # train_batch_size=8,
    # eval_batch_size=8,
    # num_train_epochs=10,

    n_gpu=1 if USE_GPU else 0,
    early_stop_callback=False,
    fp_16=False,
    opt_level='O1',
    max_grad_norm=1.0,
    seed=42,
)


## TSVデータセットクラス

TSV形式のファイルをデータセットとして読み込みます。  
形式は"{title}\t{body}\t{genre_id}"です。

In [15]:
class TsvDataset(Dataset):
    def __init__(self, tokenizer, data_dir, type_path, input_max_len=512, target_max_len=512):
        self.file_path = os.path.join(data_dir, type_path)
        
        self.input_max_len = input_max_len
        self.target_max_len = target_max_len
        self.tokenizer = tokenizer
        self.inputs = []
        self.targets = []

        self._build()
  
    def __len__(self):
        return len(self.inputs)
  
    def __getitem__(self, index):
        source_ids = self.inputs[index]["input_ids"].squeeze()
        target_ids = self.targets[index]["input_ids"].squeeze()

        source_mask = self.inputs[index]["attention_mask"].squeeze()
        target_mask = self.targets[index]["attention_mask"].squeeze()

        return {"source_ids": source_ids, "source_mask": source_mask, 
                "target_ids": target_ids, "target_mask": target_mask}

    def _make_record(self, text, chat):
        input = f"{text}"
        target = f"{chat}"
        return input, target
  
    def _build(self):
        with open(self.file_path, "r", encoding="utf-8") as f:
            for line in f:
                line = line.strip().split("\t")
                assert len(line) == 2
                assert len(line[0]) > 0
                assert len(line[1]) > 0

                text = line[0]
                chat = line[1]

                input, target = self._make_record(text, chat)

                tokenized_inputs = self.tokenizer.batch_encode_plus(
                    [input], max_length=self.input_max_len, truncation=True, 
                    padding="max_length", return_tensors="pt"
                )

                tokenized_targets = self.tokenizer.batch_encode_plus(
                    [target], max_length=self.target_max_len, truncation=True, 
                    padding="max_length", return_tensors="pt"
                )

                self.inputs.append(tokenized_inputs)
                self.targets.append(tokenized_targets)


試しにテストデータ（test.tsv）を読み込み、トークナイズ結果をみてみます。

In [16]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# トークナイザー（SentencePiece）モデルの読み込み
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL_NAME, is_fast=True)

# テストデータセットの読み込み
train_dataset = TsvDataset(tokenizer, args_dict["data_dir"], "train.tsv", 
                           input_max_len=64, target_max_len=512)


テストデータの1レコード目をみてみます。

In [17]:
test=["HElLO", "Good Morning", "THANK YOU"]
tokenizer.batch_encode_plus(test)
#大文字ローマ字が[unk]扱いになってしまい、データから消される。謎。

{'input_ids': [[447, 4818, 26571, 1], [8204, 71549, 1], [366, 184985, 38536, 1]], 'attention_mask': [[1, 1, 1, 1], [1, 1, 1], [1, 1, 1, 1]]}

In [18]:
i=0
for data in train_dataset:
  print(tokenizer.decode(data["target_ids"]))
  #print(data["target_ids"])
  i+=1
  if i > 20:
    break
#print(tokenizer.decode(train_dataset["source_ids"]))

<A> 全体をインデックスに基づいてソートし、置き換える</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pa

In [19]:
for data in train_dataset:
    print("A. 入力データの元になる文字列")
    print(tokenizer.decode(data["source_ids"]))
    print()
    print("B. 入力データ（Aの文字列がトークナイズされたトークンID列）")
    print(data["source_ids"])
    print()
    print("C. 出力データの元になる文字列")
    print(tokenizer.decode(data["target_ids"]))
    print()
    print("D. 出力データ（Cの文字列がトークナイズされたトークンID列）")
    print(data["target_ids"])
    break

A. 入力データの元になる文字列
translate English to Japanese: <A>. sort_index ( inplace = True )</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

B. 入力データ（Aの文字列がトークナイズされたトークンID列）
tensor([37194,  5413,   288, 30865,   267,  1042,   357,   669,   259,   260,
         9019,   290,  2358,   274,   281, 24657,   259,   349, 38583,   259,
          271,     1,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0])

C. 出力データの元になる文字列
<A> 全体をインデックスに基づいてソートし、置き換える</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad

## 学習処理クラス

[PyTorch-Lightning](https://github.com/PyTorchLightning/pytorch-lightning)を使って学習します。

PyTorch-Lightningとは、機械学習の典型的な処理を簡潔に書くことができるフレームワークです。

In [20]:
class T5FineTuner(pl.LightningModule):
    def __init__(self, hparams):
        super().__init__()
        self.hparams = hparams

        # 事前学習済みモデルの読み込み
        self.model = T5ForConditionalGeneration.from_pretrained(hparams.model_name_or_path)

        # トークナイザーの読み込み
        self.tokenizer = AutoTokenizer.from_pretrained(hparams.tokenizer_name_or_path, is_fast=True)

    def forward(self, input_ids, attention_mask=None, decoder_input_ids=None, 
                decoder_attention_mask=None, labels=None):
        """順伝搬"""
        return self.model(
            input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
            labels=labels
        )

    def _step(self, batch):
        """ロス計算"""
        labels = batch["target_ids"]

        # All labels set to -100 are ignored (masked), 
        # the loss is only computed for labels in [0, ..., config.vocab_size]
        labels[labels[:, :] == self.tokenizer.pad_token_id] = -100

        outputs = self(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            decoder_attention_mask=batch['target_mask'],
            labels=labels
        )

        loss = outputs[0]
        return loss

    def training_step(self, batch, batch_idx):
        """訓練ステップ処理"""
        loss = self._step(batch)
        self.log("train_loss", loss)
        return {"loss": loss}

    def validation_step(self, batch, batch_idx):
        """バリデーションステップ処理"""
        loss = self._step(batch)
        self.log("val_loss", loss)
        return {"val_loss": loss}

    def test_step(self, batch, batch_idx):
        """テストステップ処理"""
        loss = self._step(batch)
        self.log("test_loss", loss)
        return {"test_loss": loss}

    def configure_optimizers(self):
        """オプティマイザーとスケジューラーを作成する"""
        model = self.model
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in model.named_parameters() 
                            if not any(nd in n for nd in no_decay)],
                "weight_decay": self.hparams.weight_decay,
            },
            {
                "params": [p for n, p in model.named_parameters() 
                            if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]
        optimizer = AdamW(optimizer_grouped_parameters, 
                          lr=self.hparams.learning_rate, 
                          eps=self.hparams.adam_epsilon)
        self.optimizer = optimizer

        scheduler = get_linear_schedule_with_warmup(
            optimizer, num_warmup_steps=self.hparams.warmup_steps, 
            num_training_steps=self.t_total
        )
        self.scheduler = scheduler

        return [optimizer], [{"scheduler": scheduler, "interval": "step", "frequency": 1}]

    def get_dataset(self, tokenizer, type_path, args):
        """データセットを作成する"""
        return TsvDataset(
            tokenizer=tokenizer, 
            data_dir=args.data_dir, 
            type_path=type_path, 
            input_max_len=args.max_input_length,
            target_max_len=args.max_target_length)
    
    def setup(self, stage=None):
        """初期設定（データセットの読み込み）"""
        if stage == 'fit' or stage is None:
            train_dataset = self.get_dataset(tokenizer=self.tokenizer, 
                                             type_path="train.tsv", args=self.hparams)
            self.train_dataset = train_dataset

            val_dataset = self.get_dataset(tokenizer=self.tokenizer, 
                                           type_path="dev.tsv", args=self.hparams)
            self.val_dataset = val_dataset

            self.t_total = (
                (len(train_dataset) // (self.hparams.train_batch_size * max(1, self.hparams.n_gpu)))
                // self.hparams.gradient_accumulation_steps
                * float(self.hparams.num_train_epochs)
            )

    def train_dataloader(self):
        """訓練データローダーを作成する"""
        return DataLoader(self.train_dataset, 
                          batch_size=self.hparams.train_batch_size, 
                          drop_last=True, shuffle=True, num_workers=4)

    def val_dataloader(self):
        """バリデーションデータローダーを作成する"""
        return DataLoader(self.val_dataset, 
                          batch_size=self.hparams.eval_batch_size, 
                          num_workers=4)


# 転移学習を実行

GPUのOut Of Memoryエラーが発生することがあります。
その場合、次の順にハイパーパラメータの調整を試してみるとエラーを解消できる場合があります。

1. 訓練時のバッチサイズ train_batch_size を小さくする（例えば4）。  
小さくしすぎると精度が悪化することがあります。
2. 入力文の最大トークン数 max_input_length や出力文の最大トークン数 max_target_length を小さくする（例えば、入力を256や出力を32にする）。  
入力文の最大トークン数を小さくすると一般に精度が落ち、出力文の最大トークン数を小さくすると生成できる文章の長さが短くなります。

In [21]:
# 学習に用いるハイパーパラメータを設定する
args_dict.update({
    "max_input_length":  32,  # 入力文の最大トークン数
    "max_target_length": 256,  # 出力文の最大トークン数
    "train_batch_size":  2, #4
    "eval_batch_size":   4, #8
    "num_train_epochs":  7,
    })
args = argparse.Namespace(**args_dict)

train_params = dict(
    accumulate_grad_batches=args.gradient_accumulation_steps,
    gpus=args.n_gpu,
    max_epochs=args.num_train_epochs,
    precision= 16 if args.fp_16 else 32,
    amp_level=args.opt_level,
    gradient_clip_val=args.max_grad_norm,
)

In [ ]:
# 転移学習の実行（GPUを利用すれば1エポック10分程度）
model = T5FineTuner(args)
trainer = pl.Trainer(**train_params)
trainer.fit(model)

# 最終エポックのモデルを保存
model.tokenizer.save_pretrained(MODEL_DIR)
model.model.save_pretrained(MODEL_DIR)

del model

GPU available: True, used: True
TPU available: None, using: 0 TPU cores

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 300 M 
-----------------------------------------------------
300 M     Trainable params
0         Non-trainable params
300 M     Total params
1,200.707 Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


# 学習済みモデルの読み込み

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import T5ForConditionalGeneration, AutoTokenizer

# トークナイザー（SentencePiece）
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR, is_fast=True)

# 学習済みモデル
trained_model = T5ForConditionalGeneration.from_pretrained(MODEL_DIR)

# GPUの利用有無
USE_GPU = torch.cuda.is_available()
if USE_GPU:
    trained_model.cuda()

# 全テストデータの入力に対する励まし文生成

In [ ]:
import textwrap
from tqdm.auto import tqdm
from sklearn import metrics

# テストデータの読み込み
test_dataset = TsvDataset(tokenizer, args_dict["data_dir"], "test.tsv", 
                          input_max_len=args.max_input_length, 
                          target_max_len=args.max_target_length)

test_loader = DataLoader(test_dataset, batch_size=8, num_workers=4)

trained_model.eval()

inputs = []
outputs = []
targets = []

for batch in tqdm(test_loader):
    input_ids = batch['source_ids']
    input_mask = batch['source_mask']
    if USE_GPU:
        input_ids = input_ids.cuda()
        input_mask = input_mask.cuda()

    output = trained_model.generate(input_ids=input_ids, 
        attention_mask=input_mask, 
        max_length=args.max_target_length,
        repetition_penalty=10.0,   # 同じ文の繰り返し（モード崩壊）へのペナルティ
        )

    output_text = [tokenizer.decode(ids, skip_special_tokens=True, 
                            clean_up_tokenization_spaces=False) 
                for ids in output]
    target_text = [tokenizer.decode(ids, skip_special_tokens=True, 
                               clean_up_tokenization_spaces=False) 
                for ids in batch["target_ids"]]
    input_text = [tokenizer.decode(ids, skip_special_tokens=True, 
                               clean_up_tokenization_spaces=False) 
                for ids in input_ids]

    inputs.extend(input_text)
    outputs.extend(output_text)
    targets.extend(target_text)
    

## 生成結果確認

形式
- input: 入力文
- generated: 生成された励まし文
- actual: 人が作成した励まし文（正解）


In [ ]:
for output, target, input in zip(outputs, targets, inputs):
    print("input:     " + input)
    print("generated: " + output)
    print("actual:    " + target)
    print()

# タイトルに合ったニュース記事本文を生成

タイトルに合う記事を自動生成してみます。

以下のコードでは生成される文章の変な繰り返しを防いだりするために色々generateメソッドのパラメータを設定しています。パラメータの詳細は下記リンク先を参照してください。

- [generateメソッドのパラメータの意味](https://huggingface.co/transformers/main_classes/model.html#transformers.generation_utils.GenerationMixin.generate)

In [ ]:
# text="難しくて何からやればいいのかわかりません"

In [ ]:
# MAX_SOURCE_LENGTH = args.max_input_length   # 入力される記事本文の最大トークン数
# MAX_TARGET_LENGTH = args.max_target_length  # 生成されるタイトルの最大トークン数


# def preprocess_text(text):
#     return normalize_text(text.replace("\n", " "))

# def hagemasu(text):
#   # 推論モード設定
#   trained_model.eval()

#   # 前処理とトークナイズを行う
#   inputs = [preprocess_text(text)]

#   batch = tokenizer.batch_encode_plus(
#       inputs, max_length=MAX_SOURCE_LENGTH, truncation=True, 
#       padding="longest", return_tensors="pt")

#   input_ids = batch['input_ids']
#   input_mask = batch['attention_mask']
#   if USE_GPU:
#       input_ids = input_ids.cuda()
#       input_mask = input_mask.cuda()

#   # 生成処理を行う
#   outputs = trained_model.generate(
#       input_ids=input_ids, attention_mask=input_mask, 
#       max_length=MAX_TARGET_LENGTH,
#       #temperature=1.0,          # 生成にランダム性を入れる温度パラメータ
#       #num_beams=10,             # ビームサーチの探索幅
#       #diversity_penalty=1.0,    # 生成結果の多様性を生み出すためのペナルティ
#       #num_beam_groups=10,       # ビームサーチのグループ数
#       #num_return_sequences=10,  # 生成する文の数
#       repetition_penalty=1.5,   # 同じ文の繰り返し（モード崩壊）へのペナルティ
#   )

#   # 生成されたトークン列を文字列に変換する
#   generated_titles = [tokenizer.decode(ids, skip_special_tokens=True, 
#                                       clean_up_tokenization_spaces=False) 
#                       for ids in outputs]

#   # 生成された励まし文を表示する
#   #return generated_titles
#   output="".join(generated_titles)

#   return output


# hagemasu(text)


#モデルの圧縮保存
圧縮したモデルを使って[ここ](https://colab.research.google.com/drive/1-zRtFJWgo3FU3zGfr-a0SFWHJu2DQIPt?hl=ja#scrollTo=x_alXyqC42BT)で時短ボットが作れる。


In [ ]:
#モデルを圧縮保存する
#
!zip -r mT5_japy_DS.zip model